# सत्र 1 – चॅट बूटस्ट्रॅप (फाउंड्री लोकल)

हे नोटबुक फाउंड्री लोकल बूटस्ट्रॅप करते, प्राधान्य दिलेला मॉडेल उपनाम डाउनलोड करते आणि मानक तसेच स्ट्रीमिंग चॅट पूर्णता करते.


# परिस्थिती
या सत्रात तुम्हाला Foundry Local च्या माध्यमातून स्थानिक लहान भाषा मॉडेल प्रतिसाद देण्यासाठी आवश्यक किमान गोष्टींची ओळख करून दिली जाईल. तुम्ही हे कराल:
- SDK / क्लायंट अवलंबित्व स्थापित करा.
- निवडलेल्या उपनामासाठी (डिफॉल्ट: `phi-4-mini`) Foundry Local व्यवस्थापक प्रारंभ करा.
- मॉडेल मेटाडेटामधील पर्यायी फील्ड सहन करण्यासाठी एक संरक्षणात्मक मंकी-पॅच लागू करा.
- एक मानक चॅट पूर्णता विनंती पाठवा.
- प्रत्युत्तर टोकन-बाय-टोकन प्रवाहित करा.

उद्दिष्ट म्हणजे RAG, रूटिंग किंवा एजंट्सकडे जाण्यापूर्वी तुमच्या स्थानिक रनटाइम आणि नेटवर्क मार्गाची पडताळणी करणे.


### स्पष्टीकरण: अवलंबित्व स्थापना
या मिनिमल चॅट फ्लो साठी आवश्यक असलेल्या Python पॅकेजेसची स्थापना:
- `foundry-local-sdk`: स्थानिक मॉडेल्स आणि सेवा जीवनचक्र व्यवस्थापित करण्यासाठी.
- `openai`: चॅट पूर्णतेसाठी परिचित क्लायंट अब्स्ट्रॅक्शन.
- `rich`: नोटबुक आउटपुट अधिक स्पष्ट करण्यासाठी सुंदर प्रिंटिंग.

पुन्हा चालवणे सुरक्षित आहे (idempotent). जर तुमच्या वातावरणात आधीच हे पॅकेजेस असतील तर हे टाळा.


In [1]:
# Install required libraries (idempotent)
%pip install -q foundry-local-sdk openai rich


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### स्पष्टीकरण: मुख्य आयात
नोटबुकमध्ये वापरल्या जाणाऱ्या मॉड्यूल्स आणते:
- `FoundryLocalManager` स्थानिक मॉडेल रनटाइमशी संवाद साधण्यासाठी.
- `OpenAI` क्लायंट जेणेकरून आपण परिचित चॅट पूर्णता API पृष्ठभाग पुन्हा वापरू शकतो.
- `rich.print` स्टाइल केलेल्या आउटपुटसाठी.

येथे कोणतेही नेटवर्क कॉल होत नाहीत—हे फक्त नेमस्पेस तयार करते.


In [2]:
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

### स्पष्टीकरण: व्यवस्थापक प्रारंभ आणि मेटाडेटा पॅच
निवडलेल्या उपनामासाठी `FoundryLocalManager` प्रारंभ करतो आणि `promptTemplate` `null` असू शकतो अशा सेवा प्रतिसादांना सौम्यपणे हाताळण्यासाठी एक संरक्षणात्मक मंकी-पॅच लागू करतो.

मुख्य परिणाम:
- सेवा स्थिती आणि एंडपॉइंटची पुष्टी करतो.
- कॅश केलेले मॉडेल सूचीबद्ध करतो (स्थानिक स्टोअरची पडताळणी करतो).
- उपनामासाठी ठोस मॉडेल आयडी निश्चित करतो (नंतरच्या चॅट कॉल्समध्ये वापरले जाते).

जर तुम्हाला कच्च्या सेवा मेटाडेटामध्ये पडताळणी समस्या आढळल्या, तर हे पॅटर्न SDK फॉर्क न करता कसे स्वच्छ करावे हे दाखवते.


In [3]:
# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:50262/v1

Cached models:
[
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-generic-gpu:4',
        version='4',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-generic-gpu/versions/4',
        file_size_mb=3809,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-generic-gpu:3',
        version='3',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-generic-gpu/versions/3',
        file_size_mb=700,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-generic-gpu:1',
        version='1',
        execution_provider='WebGpuExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-generic-gpu/versions/1',
        file_size_mb=2211,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    )
]

Using model id: Phi-4-mini-instruct-generic-gpu:4

### स्पष्टीकरण: बेसिक चॅट पूर्णता
स्थानिक Foundry एन्डपॉइंटकडे निर्देशित `OpenAI`-सुसंगत क्लायंट तयार करते आणि एकच नॉन-स्ट्रीमिंग चॅट पूर्णता करते. येथे लक्ष केंद्रित करा:
- मॉडेल त्रुटीशिवाय प्रतिसाद देते याची खात्री करा.
- विलंब / आउटपुट स्वरूप सत्यापित करा.
- संसाधने जतन करण्यासाठी `max_tokens` माफक ठेवा.

जर हे अयशस्वी झाले, तर Foundry स्थानिक सेवा चालू आहे आणि उपनाम योग्यरित्या निराकरण करते याची पुन्हा तपासणी करा.


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':prompt}],
    max_tokens=120,
    temperature=0.5
)
print(resp.choices[0].message.content)

Local inference for privacy refers to the practice of performing data analysis on a local device without sending 
sensitive information to a central server. Two benefits of this approach are:


1. **Enhanced Privacy**: Local inference keeps personal data on the user's device, reducing the risk of data 
breaches and unauthorized access. Since the data is not transmitted over the network, it is less susceptible to 
interception by malicious actors.


2. **Data Sovereignty**: Users retain control over their data, as it does not leave their device. This means that 
individuals or organizations can comply with local data protection regulations, such as the General

### स्पष्टीकरण: स्ट्रीमिंग चॅट पूर्णता  
उत्तम जाणवणारी विलंबता आणि परस्परसंवादी UX सुधारण्यासाठी टोकन स्ट्रीमिंग दाखवते. लूप येणाऱ्या टप्प्याटप्प्याने डेल्टास प्रिंट करते:  
- अशा चॅट UI साठी उपयुक्त जिथे सुरुवातीचे अपूर्ण आउटपुट महत्त्वाचे असते.  
- तुम्हाला टोकन थ्रूपुट आणि पूर्ण पूर्णतेच्या विलंबाचा अंदाज घेण्याची परवानगी देते.  

तुम्ही या पद्धतीला टोकन जमा करण्यासाठी, प्रगती विजेट अपडेट करण्यासाठी किंवा मध्य-निर्मिती रद्द करण्यासाठी अनुकूल करू शकता.  


In [5]:
# Streaming example
stream = client.chat.completions.create(
    model=model_id,
    messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}],
    stream=True,
    max_tokens=60,
    temperature=0.4
)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

latency

and

bandwidth

usage

.

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात ठेवा की स्वयंचलित भाषांतरे त्रुटी किंवा अचूकतेच्या अभावाने युक्त असू शकतात. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून निर्माण झालेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
